# Logistic Regression - SKLearn, TensorFlow, Keras, PyTorch

Author: Harry Yau

Date: September 23, 2019

Updated: September 24, 2019

This goal of this notebook is to compute a multi-class Logistic Regression using different frameworks. The frameworks that will be investigated are:
- Sklearn
- Statsmodel
- TensorFlow 1.0
- Keras
- PyTorch
- TensorFlow 2.0

Equations pertaining to logistic regression:
$$ z = W^Tx + b$$
$$ \hat{y} = \sigma(W^Tx + b)$$
$$ \sigma(z) = \frac{1}{1+e^{-z}}$$

For multi-classification problem, the softmax function will be used for the activation function in the output layer.
$$Softmax(x_i)=\frac{e^{x_i}}{\sum_j e^{x_j}}$$

The loss function that will be used will be the cross-entropy loss.
$$H(y, \hat{y}) = - \sum_i y_i log(\hat{y_i})$$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

The numpy array is forced to have dtype np.float32 so that pytorch will accept these values.

In [2]:
X = np.array(load_iris().data, dtype=np.float32)
y = np.array(load_iris().target, dtype=np.float32)

Scaling the data so that it helps with convergence when running gradient descent in TensorFlow, Keras and Pytorch.

In [3]:
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler()
X_scaled = X_scaler.fit_transform(X)

The input variables

In [4]:
load_iris().feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

The output variables

In [5]:
load_iris().target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

### Standardized Settings for TensorFlow, Keras and PyTorch

In [6]:
input_dim = 4        # takes variable 'x' 4 input variables, due to 4 columns
output_dim = 3       # takes variable 'y' 3 output variables, due to 3 output categorizations
learning_rate = 0.01
epochs = 1000

### Sklearn

Included a large regularization value C to offset the L2 regularization that is default in SKLearn

In [7]:
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression(C=1e8, max_iter=epochs, solver='lbfgs', multi_class='multinomial', n_jobs=-1, verbose=0)
log_model.fit(X, y.squeeze())
probas = log_model.predict_log_proba(X)
predicted = np.argmax(probas, 1)
print("Accuracy Score:", accuracy_score(y, predicted))

Accuracy Score: 0.9866666666666667


### Statsmodel

The parameters method and maxiter are tuneable in the fit argument.

In [8]:
import statsmodels.api as sm # For constants
from statsmodels.discrete.discrete_model import MNLogit

X_c = sm.add_constant(X)
log_model_sm = sm.MNLogit(y, X_c).fit(method='lbfgs', maxiter=epochs) #Fit the model
predicted = np.argmax(log_model_sm.predict(X_c), 1)
print("Accuracy Score:", accuracy_score(y, predicted))

Accuracy Score: 0.9866666666666667


### TensorFlow 1.0

In TensorFlow there is no loss function specifically for cross-entropy. However there is a loss function that combines both the Softmax and the cross-entropy loss function.

There are two path ways to implement this loss function. However, this will influence how the placeholder and cost function will be chosen.
- With One-hot
    - Placeholder: Ensure the shape to be output of dimension `[None, output_dim]`. output_dim being the number of classes
        - `y_tf = tf.placeholder(tf.float32, shape=[None, output_dim], name='y')`
    - feed_dict: Ensure that you pass data as a one-hot with dtype float32. Use `pd.getdummies()` to one-hot
        - `y_onehot = pd.get_dummies(y.squeeze())`
        - `y_onehot = np.array(y_onehot, dtype='float32')`
    - `tf.losses.softmax_cross_entropy()`, this uses the function below
    - `tf.nn.softmax_cross_entropy_with_logits_v2()` - must wrap this with `tf.reduce_mean()`
- Without one-hot
    - Placeholder: Ensure the shape to be output of dimension `[None, 1]`
        - `y_tf = tf.placeholder(tf.float32, shape=[None, 1], name='y')`
    - feed_dict: Ensure that you pass a 1-D array with dtype int32
        - `y_sparse = np.array(y, dtype='int32').reshape(-1,1)`
    - Use the loss functions with sparse in the name.
        - `tf.losses.sparse_softmax_cross_entropy()`
        - `tf.nn.sparse_softmax_cross_entropy_with_logits()` - must wrap this with `tf.reduce_mean()`

In [9]:
import tensorflow as tf
print('Version:', tf.__version__)

Version: 1.14.0


The following will demonstrate the version with one-hot.

Need to one-hot the y labels for `tf.losses.softmax_cross_entropy` to work correctly.

In [10]:
y_onehot = pd.get_dummies(y.squeeze())
y_onehot = np.array(y_onehot, dtype='float32')

Creating the placeholder for the data.

In [11]:
X_tf = tf.placeholder(tf.float32, shape=[None, input_dim], name='X')
y_tf = tf.placeholder(tf.float32, shape=[None, output_dim], name='y')

Create variables for W and b. Initializing W with Xavier/Glorot Initialization and b with zeros.

In [12]:
W_tf = tf.Variable(tf.glorot_uniform_initializer()((input_dim, output_dim)), dtype=tf.float32, name='W')
b_tf = tf.Variable(tf.zeros_initializer()((output_dim)), dtype=tf.float32, name='b')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Define the y predicted equation. 

Note: There is need to wrap the equation with `tf.nn.softmax()` because the loss function calculates the softmax. Without the wrapper is the logit, and this is passed into the loss function.

In [13]:
#WX + b
y_pred_tf = tf.add(tf.matmul(X_tf, W_tf), b_tf)

#The cost function
cost_tf = tf.losses.softmax_cross_entropy(onehot_labels=y_onehot, logits=y_pred_tf)

#The optimizer
optimizer_tf = tf.train.AdamOptimizer(learning_rate).minimize(cost_tf)

#Global Variables Initilizer -- Mandatory
init = tf.global_variables_initializer()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Run the computation graph. For the output, run an argmax function `tf.argmax()` on the output results to get the categories.

In [14]:
#The Tensorflow Session
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        
        feed_dict = feed_dict={X_tf: X_scaled, y_tf: y_onehot}
        
        #Run the optimizer at every epoch
        sess.run(optimizer_tf, feed_dict)
            
        if (epoch + 1) % 50 == 0:
            #Calculate the cost at every epoch
            print("Epoch", (epoch + 1), ": cost =", cost_tf.eval(feed_dict)) 
    
    # Store values
    predicted = sess.run(tf.argmax(y_pred_tf, 1), feed_dict)

Epoch 50 : cost = 0.38219124
Epoch 100 : cost = 0.29272097
Epoch 150 : cost = 0.23531395
Epoch 200 : cost = 0.19645964
Epoch 250 : cost = 0.16920346
Epoch 300 : cost = 0.14931485
Epoch 350 : cost = 0.13427193
Epoch 400 : cost = 0.12254125
Epoch 450 : cost = 0.11315745
Epoch 500 : cost = 0.105489746
Epoch 550 : cost = 0.09911152
Epoch 600 : cost = 0.09372509
Epoch 650 : cost = 0.08911712
Epoch 700 : cost = 0.08513096
Epoch 750 : cost = 0.08164919
Epoch 800 : cost = 0.07858214
Epoch 850 : cost = 0.07586017
Epoch 900 : cost = 0.07342846
Epoch 950 : cost = 0.07124324
Epoch 1000 : cost = 0.069269195


In [15]:
accuracy_score(y, predicted)

0.98

### Keras

In Keras the loss function

In [16]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [17]:
print(tf.keras.__version__)

2.2.4-tf


In [18]:
model = Sequential([
    Dense(output_dim, activation='softmax', input_shape=(input_dim,)),
])

optimizer = tf.keras.optimizers.Adam(learning_rate)

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 15        
Total params: 15
Trainable params: 15
Non-trainable params: 0
_________________________________________________________________


In [19]:
history = model.fit(x=X_scaled, y=y, epochs=epochs, verbose=0)

In [20]:
acc = model.evaluate(X_scaled, y, verbose=0)[1]
print('Accuracy:', acc)

Accuracy: 0.98


### PyTorch

In [21]:
import torch

In [22]:
class LogisticRegressionModel_PyT(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel_PyT, self).__init__()
        self.W = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty(input_dim, output_dim)))
        self.b = torch.nn.Parameter(torch.zeros(output_dim))

        
    def forward(self, x):
        out = torch.matmul(x, self.W) + self.b
        return out            

class LogisticRegressionModel_PyT_Layers(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel_PyT_Layers, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.linear(x)
        return out            

In [23]:
model = LogisticRegressionModel_PyT(input_dim, output_dim)

if torch.cuda.is_available():
    model.cuda()
    
criterion = torch.nn.CrossEntropyLoss() #This calculates both nn.LogSoftmax and nn.NLLLoss
optimiser = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [24]:
#Convert to Variable

if torch.cuda.is_available():
    X_data = torch.from_numpy(X_scaled).cuda()
    y_data = torch.tensor(y.squeeze(), dtype=torch.long).cuda()
else:
    X_data = torch.from_numpy(X_scaled)
    y_data = torch.tensor(y.squeeze(), dtype=torch.long)

    
for epoch in range(epochs):
    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, 
    # dont want to cummulate gradients
    optimiser.zero_grad()

    # Get output from the model, given the inputs
    y_pred = model(X_data)
    # Get loss for the predicted output
    loss = criterion(y_pred, y_data)
    # Get gradients w.r.t to parameters
    loss.backward()

    # Update parameters
    optimiser.step()
    if (epoch + 1) % 50 == 0:
        print('epoch {}, loss {}'.format(epoch + 1, loss.item()))    
    

epoch 50, loss 0.3800296485424042
epoch 100, loss 0.2913043797016144
epoch 150, loss 0.2375224232673645
epoch 200, loss 0.19984683394432068
epoch 250, loss 0.1726667284965515
epoch 300, loss 0.15250207483768463
epoch 350, loss 0.1371167004108429
epoch 400, loss 0.12506866455078125
epoch 450, loss 0.11541444063186646
epoch 500, loss 0.10752244293689728
epoch 550, loss 0.10095907747745514
epoch 600, loss 0.09541921317577362
epoch 650, loss 0.0906829759478569
epoch 700, loss 0.08658842742443085
epoch 750, loss 0.08301404863595963
epoch 800, loss 0.07986698299646378
epoch 850, loss 0.07707515358924866
epoch 900, loss 0.07458183169364929
epoch 950, loss 0.07234172523021698
epoch 1000, loss 0.0703183189034462


In [25]:
with torch.no_grad():
    if torch.cuda.is_available():
        predicted = model(torch.from_numpy(X_scaled).cuda())
        _, predicted = torch.max(predicted.data, 1)
        predicted = predicted.cpu().data.numpy()
    else:
        predicted = model(torch.from_numpy(X_scaled))
        _, predicted = torch.max(predicted.data, 1)
        predicted = predicted.data.numpy()

In [26]:
accuracy_score(y, predicted)

0.9733333333333334

## Note: The environment has been switched at this point of the notebook

### TensorFlow 2.0

In [9]:
import tensorflow as tf
print('Version:', tf.__version__)
print('Version:', tf.keras.__version__)

Version: 2.0.0-rc2
Version: 2.2.4-tf


Creating the model.

Differences from TensorFlow 1.0:
- `tf.placeholders()`, `tf.Session()` has been removed
- `tf.glorot_uniform_initializer()` is gone, and has become `tf.keras.initializers.GlorotUniform()`
- Loss functions are streamlined. Must have a softmax layer and use a Cross-Entropy loss function. There are no loss function that calculates both the softmax and cross-entropy anymore.

Alternatively, the model class can utilize the layers in Keras. This is analogous to PyTorch.

In [10]:
class LogisticRegressionModel_TF2(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel_TF2, self).__init__()
        self.W = tf.Variable(tf.keras.initializers.GlorotUniform()((input_dim, output_dim)), dtype=tf.float32, name='W')
        self.b = tf.Variable(tf.zeros_initializer()((output_dim)), dtype=tf.float32, name='b')
    
    #In PyTorch the function will be 'forward' instead.
    def call(self, x):
        x = tf.matmul(x, self.W) + self.b
        out = tf.keras.activations.softmax(x)
        return out
    
class LogisticRegressionModel_TF2_Layers(tf.keras.Model):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel_TF2_Layers, self).__init__()
        self.d1 = tf.keras.layers.Dense(output_dim, activation='softmax', input_dim=(input_dim,))
        
    def call(self, x):
        x = self.d1(x)
        return x

In [11]:
model = LogisticRegressionModel_TF2(input_dim, output_dim)

In [12]:
@tf.function
def compute_loss(model, inputs, targets):
    predictions = model(inputs)
    return criterion(targets, predictions)

@tf.function
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss_value = criterion(targets, predictions)
        
    #This calculates teh gradient
    return tape.gradient(loss_value, model.trainable_variables)

criterion = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate)

In [13]:
for epoch in range(epochs):
    
    #Backward propagation
    grads = grad(model, X_scaled, y)
    
    #update parameters
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    if(epoch+1) % 50 == 0:
        template = 'Epoch {}, Loss: {}'
        print(template.format(epoch+1, compute_loss(model, X_scaled, y)))

Epoch 50, Loss: 0.5457875728607178
Epoch 100, Loss: 0.37291446328163147
Epoch 150, Loss: 0.30826982855796814
Epoch 200, Loss: 0.2659653127193451
Epoch 250, Loss: 0.23418797552585602
Epoch 300, Loss: 0.20936702191829681
Epoch 350, Loss: 0.18952356278896332
Epoch 400, Loss: 0.17334893345832825
Epoch 450, Loss: 0.15994036197662354
Epoch 500, Loss: 0.14866115152835846
Epoch 550, Loss: 0.13905280828475952
Epoch 600, Loss: 0.13077835738658905
Epoch 650, Loss: 0.1235850527882576
Epoch 700, Loss: 0.11727989464998245
Epoch 750, Loss: 0.11171285063028336
Epoch 800, Loss: 0.10676562041044235
Epoch 850, Loss: 0.1023435965180397
Epoch 900, Loss: 0.09837018698453903
Epoch 950, Loss: 0.09478282183408737
Epoch 1000, Loss: 0.09152979403734207


In [14]:
predicted = tf.argmax(model(X_scaled), 1)
accuracy_score(y, predicted)

0.9733333333333334